The STM32 SVD files are located at:

\<CubeIDE Directory\>\STM32CubeIDE\plugins\com.st.stm32cube.ide.mcu.productdb.debug_xxxxxx\resources\cmsis\STMicroelectronics_CMSIS_SVD\

Copy the `.SVD` file to the same directory as this notebook.

Change the `model` according to the model.

Then run all cells, and a python file will be generated, named by the model. Import it in your application code.

In [1]:
model = "g474"

In [2]:
import xml.etree.ElementTree as ET
import re
import copy

In [3]:
def get_description(node):
    if node.find("description") is None:
        desc = ""
    else:
        desc = node.find("description").text
        desc = desc.replace("\n", ". ", 1)
    return " ".join(desc.split())

In [4]:
root = ET.parse(f"./stm32{model}.svd").getroot().find("peripherals")
for child in root:
    print(child.tag, child.find("name").text, child.attrib)

peripheral CRC {}
peripheral IWDG {}
peripheral WWDG {}
peripheral I2C1 {}
peripheral I2C2 {'derivedFrom': 'I2C1'}
peripheral I2C3 {'derivedFrom': 'I2C1'}
peripheral I2C4 {'derivedFrom': 'I2C1'}
peripheral FLASH {}
peripheral DBGMCU {}
peripheral RCC {}
peripheral PWR {}
peripheral RNG {}
peripheral GPIOA {}
peripheral GPIOB {}
peripheral GPIOC {}
peripheral GPIOD {'derivedFrom': 'GPIOC'}
peripheral GPIOE {'derivedFrom': 'GPIOC'}
peripheral GPIOF {'derivedFrom': 'GPIOC'}
peripheral GPIOG {'derivedFrom': 'GPIOC'}
peripheral TIM15 {}
peripheral TIM16 {}
peripheral TIM17 {'derivedFrom': 'TIM16'}
peripheral TIM1 {}
peripheral TIM20 {'derivedFrom': 'TIM1'}
peripheral TIM8 {'derivedFrom': 'TIM1'}
peripheral TIM2 {}
peripheral TIM3 {'derivedFrom': 'TIM2'}
peripheral TIM4 {'derivedFrom': 'TIM2'}
peripheral TIM5 {'derivedFrom': 'TIM2'}
peripheral TIM6 {}
peripheral TIM7 {'derivedFrom': 'TIM6'}
peripheral LPTIMER1 {}
peripheral USART1 {}
peripheral USART2 {'derivedFrom': 'USART1'}
peripheral USA

In [5]:
info = {}

for peri in root:
    periname = peri.find("name").text
    periname_simple = re.sub(r"\d+$", "", periname)
    
    info[periname] = {}
    info[periname]["address"] = int(peri.find("baseAddress").text, base=16)

    if peri.attrib.get("derivedFrom"):
        basename = peri.attrib.get("derivedFrom")
        info[periname]["base"] = basename
        continue

    info[periname]["description"] = get_description(peri)
    regsinfo = {}
    info[periname]["registers"] = regsinfo

    regs = peri.find("registers")
    for reg in regs:
        regname = reg.find("name").text
        regname_simple = re.sub(f"^({periname}_|{periname}|{periname_simple}_|{periname_simple})", "", regname)
        if re.findall(r"^\d", regname_simple):
            regname = regname
        else:
            regname = regname_simple
        regsinfo[regname] = {}
        regsinfo[regname]["description"] = get_description(reg)
        regsinfo[regname]["addressOffset"] = int(reg.find("addressOffset").text, base=16)
        reg_access = reg.find("access")
        if reg_access is None:
            reg_access = ""
        else:
            reg_access = reg_access.text
        regsinfo[regname]["resetValue"] = int(reg.find("resetValue").text, base=16)
        fieldsinfo = {}
        regsinfo[regname]["fields"] = fieldsinfo
        
        fields = reg.find("fields")
        if fields is None:
            continue
        for field in fields:
            finfo = {}
            fname = field.find("name").text
            fieldsinfo[fname] = finfo
            finfo["description"] = get_description(field)
            offset = int(field.find("bitOffset").text)
            width = int(field.find("bitWidth").text)
            finfo["mask"] = (2**width - 1) * 2**offset
            field_access = field.find("access")
            if field_access is None:
                field_access = ""
            else:
                field_access = field_access.text
            finfo["access"] = reg_access + field_access

        toremove = set()
        for fname, finfo in fieldsinfo.items():
            if re.findall(r"(_\d+)+$", fname):
                fname_simple = re.sub(r"(_\d+)+$", "", fname)
                if fname_simple in fieldsinfo:
                    fieldsinfo[fname_simple]["mask"] |= finfo["mask"]
                    toremove.add(fname)
        for fname in toremove:
            fieldsinfo.pop(fname)

    tomerge = {}
    for regname, reginfo in regsinfo.items():
        if re.findall("(_Input|_Output)$", regname):
            regname_simple = re.sub("(_Input|_Output)$", "", regname)
            if regname_simple in tomerge:
                tomerge[regname_simple]["fields"] |= regsinfo[regname]["fields"]
            else:
                tomerge[regname_simple] = copy.deepcopy(regsinfo[regname])
                tomerge[regname_simple]["description"] = re.sub(r"\([^)]*\)", "",
                    tomerge[regname_simple]["description"]).rstrip(" ")
    regsinfo |= tomerge

info

{'CRC': {'address': 1073885184,
  'description': 'Cyclic redundancy check calculation unit',
  'registers': {'DR': {'description': 'Data register',
    'addressOffset': 0,
    'resetValue': 4294967295,
    'fields': {'DR': {'description': 'Data register bits',
      'mask': 4294967295,
      'access': 'read-write'}}},
   'IDR': {'description': 'Independent data register',
    'addressOffset': 4,
    'resetValue': 0,
    'fields': {'IDR': {'description': 'General-purpose 8-bit data register bits',
      'mask': 4294967295,
      'access': 'read-write'}}},
   'CR': {'description': 'Control register',
    'addressOffset': 8,
    'resetValue': 0,
    'fields': {'REV_OUT': {'description': 'Reverse output data',
      'mask': 128,
      'access': 'read-write'},
     'REV_IN': {'description': 'Reverse input data',
      'mask': 96,
      'access': 'read-write'},
     'POLYSIZE': {'description': 'Polynomial size',
      'mask': 24,
      'access': 'read-write'},
     'RESET': {'description': '

In [6]:
def find_arrays(ls):
    results = set()
    re_last_digits = r"(\d+)(?!.*\d)"

    for s in ls:
        if re.findall(re_last_digits, s):
            n = int(re.findall(re_last_digits, s)[0])
            fmt = re.sub(re_last_digits, "{}", s)
            if fmt not in results and \
                fmt.format("") not in ls and \
                fmt.format(n+1) in ls and\
                fmt != "CR{}":
                results.add(fmt)
    
    return results

In [7]:
f = open(f"./{model}.py", "w")
f.write("from seracc import BitField, RegisterBase, PeripheralBase, Subscriptor\n")
f.write("from seracc import logging, wait_until_equal\n")

45

In [8]:
for periname, periinfo in info.items():
    if "base" in periinfo:
        f.write(f"""\
{periname} = SA_{periinfo["base"]}(0x{periinfo["address"]:08X}, \"{periname}\")
""")
        continue

    peri_class = f"""
class SA_{periname}(PeripheralBase):

    def __init__(self, base, name):
        super().__init__(base, name, \"{periinfo["description"]}\")
"""
    
    for regname, reginfo in periinfo["registers"].items():
        classname = f"{periname}_{regname}"
        offset = reginfo["addressOffset"]
        peri_class += f"""\
        self.{regname} = SA_{classname}(self, 0x{offset:X})
"""
        
        reg_class = f"""
class SA_{classname}(RegisterBase):
    
    def __init__(self, peripheral, offset):
        super().__init__(peripheral, offset, 0x{reginfo["resetValue"]:X}, \"{regname}\", \"{reginfo["description"]}\")
"""
        for fieldname, fieldinfo in reginfo["fields"].items():
            reg_class += f"""\
        self.{fieldname} = BitField(self, 0x{fieldinfo["mask"]:08X}, \"{fieldname}\", \"{fieldinfo["description"]}\")
"""
        
        for format in find_arrays(reginfo["fields"].keys()):
            reg_class += f"""\
        self.{format.format("")} = Subscriptor(self, \"{format}\")
"""
        
        f.write(reg_class)

    for format in find_arrays(periinfo["registers"].keys()):
        peri_class += f"""\
        self.{format.format("")} = Subscriptor(self, \"{format}\")
"""
    
    peri_class += f"""
{periname} = SA_{periname}(0x{periinfo["address"]:08X}, \"{periname}\")
"""
    f.write(peri_class)

In [9]:
f.close()